In [15]:
import streamlit as st
import pandas as pd
import numpy as np
from csv import DictWriter
import psycopg2
import base64
import pendulum
import datetime
import os

In [2]:
clows = pd.read_csv('couplb.csv')
clows.head()

,Species,Location,Name,Markings,Visited
0,Cat,UP Gate,Up-up,"Pure orange, red collar, notch",Not Visited
1,Cat,UP Gate,Tigger,NaN,Not Visited
2,Cat,UP Gate,Charito,NaN,Not Visited
3,Cat,UP Gate,Izzy,NaN,Not Visited
4,Cat,UP Gate,Nessie,NaN,Not Visited


In [5]:
clowders = clows.Location.unique().tolist()
clowders

['UP Gate',
 'Admin-CPark',
 'ICOPED',
 'OUR',
 'MSI',
 'PhySci',
 'Library',
 'IBS',
 'IH-GS',
 'SU',
 'Baker',
 'WRH',
 'MRH',
 'YMCA',
 'CEC',
 'CVM',
 'FC',
 'ATI-FC',
 'ATI',
 'FOREHA',
 'MAREHA',
 'FSB',
 'DMST',
 'FPark',
 'Copeland',
 'NCPC']

In [127]:
data = [['2022-09-19','OUR','Max'], 
        ['2022-09-19','UP Gate','Alex'],
        ['2022-09-19','Admin-CPark','Jen'], 
        ['2022-09-19','MSI','Dave'],
        ['2022-09-19','PhySci','Haley'], 
        ['2022-09-19','Library','Jessie'],
        ['2022-09-19','IBS','Don'], 
        ['2022-09-19','IH-GS','Don'],
        ['2022-09-20','OUR','Don'], 
        ['2022-09-20','FSB','Adam'],
        ['2022-09-20','Admin-CPark','Park'], 
        ['2022-09-20','MSI','Max'],
        ['2022-09-20','DMST','Max'], 
        ['2022-09-20','Library','Max'],
        ['2022-09-20','Copeland','Max'], 
        ['2022-09-20','NCPC','Max']]
df = pd.DataFrame(data, columns=['Timestamp','clowder','feeder'])

In [128]:
df

,Timestamp,clowder,feeder
0,2022-09-19,OUR,Max
1,2022-09-19,UP Gate,Alex
2,2022-09-19,Admin-CPark,Jen
3,2022-09-19,MSI,Dave
4,2022-09-19,PhySci,Haley
5,2022-09-19,Library,Jessie
6,2022-09-19,IBS,Don
7,2022-09-19,IH-GS,Don
8,2022-09-20,OUR,Don
9,2022-09-20,FSB,Adam


In [129]:
today = pendulum.now()
start = today.start_of('week').strftime('%Y-%m-%d')
end = today.end_of('week').strftime('%Y-%m-%d')

week = {'day1':0,'day2':1,'day3':2,'day4':3,'day5':4,'day6':5,'day7':6}

for day, num in week.items():
    week[day] = datetime.datetime.strptime(start, '%Y-%m-%d') + datetime.timedelta(days=num)
    week[day] = week[day].strftime('%Y-%m-%d')
week

{'day1': '2022-09-19',
 'day2': '2022-09-20',
 'day3': '2022-09-21',
 'day4': '2022-09-22',
 'day5': '2022-09-23',
 'day6': '2022-09-24',
 'day7': '2022-09-25'}

In [192]:
sched = pd.DataFrame()
for _, date in week.items():
    for clow in clowders:
        if date in df.Timestamp.unique() and clow in df.clowder.unique():
            data = df[(df['Timestamp'] == date) & (df['clowder'] == clow)].feeder.values
            if len(data) > 0:
                sched.loc[clow, date] = data[0]
            else:
                sched.at[clow, date] = 'unassigned'
        else:
            sched.at[clow, date] = 'unassigned'

In [193]:
sched

,2022-09-19,2022-09-20,2022-09-21,2022-09-22,2022-09-23,2022-09-24,2022-09-25
UP Gate,Alex,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
Admin-CPark,Jen,Park,unassigned,unassigned,unassigned,unassigned,unassigned
ICOPED,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
OUR,Max,Don,unassigned,unassigned,unassigned,unassigned,unassigned
MSI,Dave,Max,unassigned,unassigned,unassigned,unassigned,unassigned
PhySci,Haley,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
Library,Jessie,Max,unassigned,unassigned,unassigned,unassigned,unassigned
IBS,Don,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
IH-GS,Don,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
SU,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned


In [213]:
import matplotlib
fs = pd.unique(sched.values.ravel()).tolist()
fs.remove('unassigned')

colors = dict(zip(fs,
              (f'background-color: {c}' for c in matplotlib.colors.cnames.values())))
colors['unassigned'] = 'background-color: gray'
for name, value in colors.items():
    if value == 'background-color: #000000':
        colors[name] = 'background-color: #FFFFFF'
        
colors

{'Alex': 'background-color: #F0F8FF',
 'Jen': 'background-color: #FAEBD7',
 'Park': 'background-color: #00FFFF',
 'Max': 'background-color: #7FFFD4',
 'Don': 'background-color: #F0FFFF',
 'Dave': 'background-color: #F5F5DC',
 'Haley': 'background-color: #FFE4C4',
 'Jessie': 'background-color: #FFFFFF',
 'Adam': 'background-color: #FFEBCD',
 'unassigned': 'background-color: gray'}

In [214]:
def style_negative(v):
    for name, color in colors.items():
        if v == 'unassigned':
            return 'background-color:grey' 
        else:
            return color

s2 = sched.style.applymap(colors.get)
s2

,2022-09-19,2022-09-20,2022-09-21,2022-09-22,2022-09-23,2022-09-24,2022-09-25
UP Gate,Alex,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
Admin-CPark,Jen,Park,unassigned,unassigned,unassigned,unassigned,unassigned
ICOPED,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
OUR,Max,Don,unassigned,unassigned,unassigned,unassigned,unassigned
MSI,Dave,Max,unassigned,unassigned,unassigned,unassigned,unassigned
PhySci,Haley,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
Library,Jessie,Max,unassigned,unassigned,unassigned,unassigned,unassigned
IBS,Don,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
IH-GS,Don,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
SU,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned


In [76]:
def highlight_survived(s):
    for i in s.values:
        if i == 'unassigned':
            return {'background-color':'lightgreen'}
        else:
            return {'background-color':'gray'}

sched.style.apply(highlight_survived, axis=0)
#st.dataframe(df.style.applymap(color_survived, subset=['Survived']))


Max
Max
unassigned
unassigned
unassigned
unassigned
unassigned


ValueError: Function <function highlight_survived at 0x0000028D57521B80> returned the wrong shape.
Result has shape: (1, 7)
Expected shape:   (26, 7)